In [3]:
import numpy as np

# 常量
ANGSTROM_TO_BOHR = 1.8897259886  # 1 Å -> bohr
EV_TO_HARTREE = 1.0 / 27.211386  # eV -> Hartree (atomic units)

def primitive_fcc(a_angstrom):
    """返回 FCC 原胞的原始矢量（以 bohr 为单位）。
    a_angstrom: 晶格常数（Å）
    返回: 3x3 ndarray，三行分别是 a1,a2,a3（bohr）
    """
    a = a_angstrom * ANGSTROM_TO_BOHR
    a1 = 0.5 * a * np.array([0.0, 1.0, 1.0])
    a2 = 0.5 * a * np.array([1.0, 0.0, 1.0])
    a3 = 0.5 * a * np.array([1.0, 1.0, 0.0])
    return np.array([a1, a2, a3])

def reciprocal_vectors(a_vecs):
    """根据实空间基矢 a1,a2,a3 计算倒格子基矢 b1,b2,b3（以 1/bohr 为单位）。
    a_vecs: 3x3 ndarray 三行是 a1,a2,a3
    返回: 3x3 ndarray 三行是 b1,b2,b3
    """
    a1, a2, a3 = a_vecs
    V = np.dot(a1, np.cross(a2, a3))
    b1 = 2.0 * np.pi * np.cross(a2, a3) / V
    b2 = 2.0 * np.pi * np.cross(a3, a1) / V
    b3 = 2.0 * np.pi * np.cross(a1, a2) / V
    return np.array([b1, b2, b3]), V

def generate_G_vectors(b_vecs, Ecut_eV, kvec=None):
    """
    生成满足动能截断的平面波 G = n1*b1 + n2*b2 + n3*b3 列表。
    b_vecs: 3x3 ndarray 倒格子三行 b1,b2,b3（1/bohr）
    Ecut_eV: 截断能（eV）
    kvec: 布里渊区内的 k 向量（以 1/bohr 为单位），默认 Gamma (0)
    返回: (Gs, coeffs, energies)
      - Gs: Nx3 ndarray，每行为一个 G 向量（1/bohr）
      - coeffs: 对应的整数系数列表 (n1,n2,n3)
      - energies: 对应的动能 0.5*|k+G|^2（Hartree）
    """
    if kvec is None:
        kvec = np.zeros(3)
    Ecut_ha = Ecut_eV * EV_TO_HARTREE
    maxG = np.sqrt(2.0 * Ecut_ha)  # 因为 T = 0.5 * |k+G|^2 <= Ecut

    norms = np.linalg.norm(b_vecs, axis=1)
    min_b = norms.min()
    # 取一个安全的整数范围，可能会包含多余项，但保证覆盖所有满足截断的 G
    Nmax = int(np.ceil(maxG / min_b)) + 1

    Gs = []
    coeffs = []
    energies = []
    for n1 in range(-Nmax, Nmax+1):
        for n2 in range(-Nmax, Nmax+1):
            for n3 in range(-Nmax, Nmax+1):
                G = n1 * b_vecs[0] + n2 * b_vecs[1] + n3 * b_vecs[2]
                kval = kvec + G
                T = 0.5 * np.dot(kval, kval)
                if T <= Ecut_ha + 1e-12:
                    Gs.append(G)
                    coeffs.append((n1, n2, n3))
                    energies.append(T)
    if len(Gs) == 0:
        return np.empty((0,3)), [], np.array([])
    Gs = np.array(Gs)
    energies = np.array(energies)
    order = np.argsort(energies)
    sorted_Gs = Gs[order]
    sorted_coeffs = [coeffs[i] for i in order]
    sorted_energies = energies[order]
    return sorted_Gs, sorted_coeffs, sorted_energies

# ----------------- 示例: Al（FCC 原胞） -----------------

# if __name__ == '__main__':
#     # 铝的实验晶格常数 (约 4.05 Å)，用户可修改
#     a0 = 4.05  # Å
#     a_vecs = primitive_fcc(a0)
#     b_vecs, _ = reciprocal_vectors(a_vecs)

#     # 截断能: 例如 300 eV（可按需调整）
#     Ecut_eV = 300.0
#     Gs, coeffs, energies = generate_G_vectors(b_vecs, Ecut_eV)

#     print(f'Crystal: Al (FCC), a = {a0} Å')
#     print(f'Ecut = {Ecut_eV} eV -> {Ecut_eV * EV_TO_HARTREE:.5f} Hartree')
#     print('Number of plane waves (Gamma):', len(Gs))
#     print('First 8 G coefficients:', coeffs[:8])
#     print('First 8 kinetic energies (Hartree):', energies[:8])
